## NIN网络代码复现

### 导入python库

In [1]:
import torch
from torch import nn
from torchinfo import summary

### 创新点
1. 引入了MLPconv结构（1 x 1卷积的使用）
2. 使用了全局平均池化（GAP）来替代全连接层

### NIN模型定义

![picture](./model2.png "picture")

![picture](./model.png "picture")

In [2]:
class NIN_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.block = nn.Sequential(nn.Conv2d(3, 192, kernel_size=5, padding=2),nn.ReLU(inplace=True),
                                   nn.Conv2d(192, 160, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.Conv2d(160, 96, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.MaxPool2d(kernel_size=3, stride=2),
                                   nn.Dropout(0.2),

                                   nn.Conv2d(96, 192, kernel_size=5, padding=2),nn.ReLU(inplace=True),
                                   nn.Conv2d(192, 192, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.Conv2d(192, 192, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.MaxPool2d(kernel_size=3, stride=2),
                                   nn.Dropout(0.2),

                                   nn.Conv2d(192, 192, kernel_size=3, padding=1),nn.ReLU(inplace=True),
                                   nn.Conv2d(192, 192, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.Conv2d(192, 10, kernel_size=1),nn.ReLU(inplace=True),
                                   nn.AvgPool2d(kernel_size=7, stride=1))

    def forward(self,x):
            x = self.block(x)
            output = torch.softmax(x,dim=1)
            return output


In [3]:
#20个维度,一个通道,图片大小
data = torch.ones(size=(20,3,32,32))

In [4]:
NIN_Model()

NIN_Model(
  (block): Sequential(
    (0): Conv2d(3, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): Conv2d(192, 160, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(160, 96, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): ReLU(inplace=True)
    (10): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Dropout(p=0.2, inplace=False)
    (16): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace=True)
    (18): Conv2d(192, 192, kernel_size=(1, 1)

In [5]:
net = NIN_Model()#实例化
output = net(data)

In [6]:
output.shape

torch.Size([20, 10, 1, 1])

### 模型信息

In [7]:
summary(net,input_size=(20,3,32,32), device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
NIN_Model                                [20, 10, 1, 1]            --
├─Sequential: 1-1                        [20, 10, 1, 1]            --
│    └─Conv2d: 2-1                       [20, 192, 32, 32]         14,592
│    └─ReLU: 2-2                         [20, 192, 32, 32]         --
│    └─Conv2d: 2-3                       [20, 160, 32, 32]         30,880
│    └─ReLU: 2-4                         [20, 160, 32, 32]         --
│    └─Conv2d: 2-5                       [20, 96, 32, 32]          15,456
│    └─ReLU: 2-6                         [20, 96, 32, 32]          --
│    └─MaxPool2d: 2-7                    [20, 96, 15, 15]          --
│    └─Dropout: 2-8                      [20, 96, 15, 15]          --
│    └─Conv2d: 2-9                       [20, 192, 15, 15]         460,992
│    └─ReLU: 2-10                        [20, 192, 15, 15]         --
│    └─Conv2d: 2-11                      [20, 192, 15, 15]         3